In [1]:
import time
import sys
import threading
from pymata4 import pymata4

In [2]:
DIGITAL_PIN = 6  # Arduino pin number
LED_PIN = 4

POLL_TIME = 5  # Number of seconds between polls

CB_PIN_MODE = 0
CB_PIN = 1
CB_VALUE = 2
CB_TIME = 3


In [4]:
def the_callback(data):
    """
    A callback function to report data changes.
    This will print the pin number, its reported value and
    the date and time when the change occurred

    :param data: [pin, current reported value, pin_mode, timestamp]
    """
    date = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(data[CB_TIME]))
    print(f'Pin: {data[CB_PIN]} Value: {data[CB_VALUE]} Time Stamp: {date}')
    board.digital_write(LED_PIN,1)
    time.sleep(1000)
    board.digital_write(LED_PIN,0)

In [6]:
def digital_in(my_board, pin):
    """
    This function establishes the pin as a
    digital input. Any changes on this pin will
    be reported through the call back function.

    :param my_board: a pymata_express instance
    :param pin: Arduino pin number
    """

    # Set the pin mode
    my_board.set_pin_mode_digital_input(pin, callback=the_callback)

    def poll_pin():
        """
        Poll the pin value and print it.
        This function will be called by the timer.
        """
        value, time_stamp = my_board.digital_read(pin)
        date = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time_stamp))
        print(f'Polling - last value: {value} received on {date}')
        # Restart the timer
        timer = threading.Timer(POLL_TIME, poll_pin)
        timer.start()
        timers.append(timer)

    # Start the initial timer
    timer = threading.Timer(POLL_TIME, poll_pin)
    timer.start()
    timers.append(timer)


In [7]:
board = pymata4.Pymata4(com_port="COM4")
board.set_pin_mode_digital_output(LED_PIN)

# List to keep track of active timers
timers = []

try:
    digital_in(board, DIGITAL_PIN)
    # Keep the main thread alive to allow timer to function
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    # Shutdown board and cancel timers on interrupt
    board.shutdown()
    for timer in timers:
        timer.cancel()
    sys.exit(0)


pymata4:  Version 1.15

Copyright (c) 2020 Alan Yorinks All Rights Reserved.

Opening COM4...

Waiting 4 seconds(arduino_wait) for Arduino devices to reset...
Arduino compatible device found and connected to COM4

Retrieving Arduino Firmware ID...
Arduino Firmware ID: 1.2 FirmataExpress.ino

Retrieving analog map...
Auto-discovery complete. Found 20 Digital Pins and 6 Analog Pins


Polling - last value: 0 received on 2024-07-17 14:05:15
Polling - last value: 0 received on 2024-07-17 14:05:15
Polling - last value: 0 received on 2024-07-17 14:05:15
Polling - last value: 0 received on 2024-07-17 14:05:15
Polling - last value: 0 received on 2024-07-17 14:05:15
Polling - last value: 0 received on 2024-07-17 14:05:15
Pin: 6 Value: 1 Time Stamp: 2024-07-17 14:05:48
Polling - last value: 1 received on 2024-07-17 14:05:48
Polling - last value: 1 received on 2024-07-17 14:05:48
Polling - last value: 1 received on 2024-07-17 14:05:48
Polling - last value: 1 received on 2024-07-17 14:05:48
Polling

: 

In [8]:
board.shutdown()

: 